In [8]:
pip install transformers datasets pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install transformers

In [2]:
!pip install accelerate

In [3]:
!pip install sentencepiece

In [1]:
# 방법 1 : 다운로드하려는 파일(페가수스)의 저장소 ID와 파일 이름 설정
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="google/pegasus-xsum", filename="config.json")

'C:\\Users\\user\\.cache\\huggingface\\hub\\models--google--pegasus-xsum\\snapshots\\8d8ffc158a3bee9fbb03afacdfc347c823c5ec8b\\config.json'

In [2]:
# 방법 2 : 다운로드 하려는 파일(페가수스)의 특정 버전을 다운로드하려면 
# revision매개변수를 사용하여 분기 이름, 태그 또는 커밋 해시를 지정해야함. 
from huggingface_hub import hf_hub_download
hf_hub_download(
    repo_id="google/pegasus-xsum", 
    filename="config.json", 
    revision="4d33b01d79672f27f001f6abade33f22d993b151")

'C:\\Users\\user\\.cache\\huggingface\\hub\\models--google--pegasus-xsum\\snapshots\\4d33b01d79672f27f001f6abade33f22d993b151\\config.json'

In [3]:
from huggingface_hub import login                      # 허깅페이스 로그인. https://huggingface.co/settings/tokens의 토큰 이용
login()

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline, AutoTokenizer
from datasets import load_dataset
import pandas as pd

# 모델 및 파이프라인 설정
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

`AnnotionFormat` is deprecated and will be removed in v4.38. Please use `transformers.image_utils.AnnotationFormat` instead.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# 모델 저장
model_save_path = "whisper_model"
model.save_pretrained(model_save_path)

# 프로세서 저장
processor_save_path = "whisper_processor"
processor.save_pretrained(processor_save_path)

## 저장된 모델 불러오기

In [5]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline, AutoTokenizer

# 저장된 모델 로드
model_save_path = "whisper_model"
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_save_path)

# 저장된 프로세서 로드
processor_save_path = "whisper_processor"
processor = AutoProcessor.from_pretrained(processor_save_path)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# 음성에서 텍스트로 변환하는 함수
def convert_audio_to_text(audio_path, pipeline):
    result = pipeline(audio_path)
    return result["text"]

# 모델 및 파이프라인 설정
device = "cuda:0" if torch.cuda.is_available() else "cpu"

torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model.to(device)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [11]:
from tqdm import tqdm

In [15]:
# 결과를 저장할 리스트
results = []

# 음성을 텍스트로 변환하고 결과를 리스트에 추가
for i in tqdm(range(31, 61), desc="Processing"):
    audio_path = f"fail_audio/fail{i}_audio.wav"
    text_result = convert_audio_to_text(audio_path, pipe)
    results.append(text_result)

# 결과를 DataFrame으로 변환
df = pd.DataFrame({"Text": results})

# CSV 파일로 저장
df.to_csv("fail_results2.csv", index=False)

# 결과 출력
print(df)

Processing: 100%|███████████████████████████████████████████████████████████████████| 30/30 [1:33:33<00:00, 187.13s/it]

                                                 Text
0    네, 시작하겠습니다. 자동차 제조사인 A사는 모델 E를 앞세워서 비극에 진출을 했...
1    취업박람회는 학생들에게 최신 취업 트렌드와 다양한 기업의 채용 정보를 제공하고 실...
2    스타트업의 해외시장 진출은 수출 증대, 경제성장 동력 확보, 일자리 창출 등에 기...
3    만약에 업무량이 많고 보고서 제출 시기 또한 앞당겨진 상황에 팀장님께서 대학 입학...
4    지원자 문 보름입니다. 저는 해외 무역 간의 지사와 담당자로서 B기업을 선택하도록...
5    사람과의 갈등 발생 시 나에게 문제가 있다면 사과하고 소통을 통해 갈등을 해결하려...
6    네, 저는 아이클린 선글라스의 진출 국가를 시국으로 선택하도록 하겠습니다. 시국을...
7    수긍하는 합리적인 사람입니다. 일단 청구된 수리비가 과다하다고 생각되기 때문에 다...
8    현지화 전략에 실패한 A사는 현지 시장에서 경쟁 우위 측면에서 성공 가능성이 큰 ...
9    사전적으로 기존 연사자들의 불참 상황을 미리 준비하였을 것입니다. 개선 요청 사항...
10   디지털 트랜스포메이션이 기업들의 지속 경영 전략으로서 전 세계적으로 활용되고 있으...
11   저는 팀장님이 직원의 손목을 잡은 것에 대해서 정확히 제가 본 장면만 진술하도록 ...
12   안녕하세요. 지원자 OO PT 발표 시작하도록 하겠습니다. 저는 우선 A기업의 사...
13   안녕하세요. 지원자 OO PT 발표 시작하도록 하겠습니다. 저는 우선 A기업의 사...
14   안녕하세요. 지원자 OO건 PT 발표 시작하도록 하겠습니다. 국내 시장에서 잘 알...
15   저는 학생회장으로서 목표를 만족도 높은 ESG 취업 박람회를 개최하고자 합니다. ...
16   시작하겠습니다. 본인의 저의 성격이 긍정적이기 때문에 카포를 선배와 좋은 관계를 ...
17   네, 발표 시작하겠습니다. 현재 상황은 코

In [14]:
pd.read_csv('fail_results.csv')

,Text
0,안녕하세요. 우리나라의 K-서비스 수출화 방안에 대한 발표를 맡은 권입니다. 현재...
1,Q. 현재 상황은? 자기 개발을 하려고 노력을 하는 성격입니다. 그래서 저라면 처...
2,안녕하세요. 지원자 권도은입니다. 저는 A 기업이 태국 시장 진출 방안에 대해서 ...
3,학술제를 준비해야 하는 상황에서 작년에 발생한 문제를 해결하고 성공적으로 학술제를...
4,안녕하세요. 지원자 권도은입니다. 현재 A사가 시국에 진출하려는 상황으로 시국의 ...
5,Q&A 관심도와 업무량을 고려를 해서 인원을 배치하겠습니다. 우선 IT 기술 적용...
6,"코트라의 핵심 사업은 사업 홈페이지에 의거하여 굉장히 다양한데 크게 무역, 투자,..."
7,우선 현재 상황으로는 시장 조사 보고서를 일주일 당겨졌기 때문에 긴급히 제출해야 ...
8,시아씨요. 발표 시작하겠습니다. 현재 상황에 따라 전자를 선택하게 될 경우 고속열...
9,제가 회사 생활을 한다면 맡은 일을 어떻게든 수행하려 노력함과 동시에 직원 간의 ...
